In [722]:
import pandas as pd
import numpy as np
import re

In [723]:
madden = pd.read_csv("maddenratings.csv")
madden = madden.rename(columns={"Full Name":"Player"})
madden.head()

,Player,Team,Position,college,awareness_rating,throwPower_rating,kickReturn_rating,leadBlock_rating,strength_rating,bCVision_rating,...,archetype,blockShedding_rating,runBlockFinesse_rating,teamId,agility_rating,fullNameForSearch,overall_rating,passBlockFinesse_rating,age,primaryKey
0,Aaron Donald,Rams,RE,Pittsburgh,99,20,10,23,99,49,...,DE_PowerRusher,97,45,24,86,Aaron Donald,99,45,31,10852
1,Davante Adams,Raiders,WR,Fresno State,99,45,45,25,63,94,...,WR_DeepThreat,52,12,23,95,Davante Adams,99,17,29,10823
2,Myles Garrett,Browns,RE,Texas AM,98,55,10,17,96,39,...,DE_PowerRusher,90,45,5,85,Myles Garrett,99,45,26,12520
3,Trent Williams,49ers,LT,Oklahoma,99,12,10,98,98,10,...,OT_Power,32,99,15,73,Trent Williams,99,92,34,9800
4,Cooper Kupp,Rams,WR,Eastern Wash.,99,59,84,34,66,96,...,WR_Slot,28,28,24,95,Cooper Kupp,98,34,29,12558


In [724]:
#this function takes in their rating and gives them a class from 1-5 (detailed in the readme)
def rating_class(rating):
    if rating < 76:
        return 1
    if rating >= 76 and rating < 83:
        return 2
    if rating >= 83 and rating < 89:
        return 3
    if rating >= 89 and rating < 95:
        return 4
    if rating >= 95:
        return 5
    
madden["class"] = list(map(rating_class, madden["overall_rating"]))

In [725]:
passing = pd.read_csv("2021-passing.csv")
passing.head()

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD
0,1,Tom Brady*,TAM,44,QB,17,17,13-4-0,485,719,...,312.7,102.1,68.1,22,144,3.0,6.98,7.41,3.0,5.0
1,2,Justin Herbert*,LAC,23,QB,17,17,9/8/2000,443,672,...,294.9,97.7,65.6,31,214,4.4,6.83,6.95,5.0,5.0
2,3,Matthew Stafford,LAR,33,QB,17,17,12/5/2000,404,601,...,287.4,102.9,63.8,30,243,4.8,7.36,7.45,3.0,4.0
3,4,Patrick Mahomes*,KAN,26,QB,17,17,12/5/2000,436,658,...,284.6,98.5,62.2,28,146,4.1,6.84,7.07,3.0,3.0
4,5,Derek Carr,LVR,30,QB,17,17,10/7/2000,428,626,...,282.6,94.0,52.4,40,241,6.0,6.85,6.60,3.0,6.0


In [726]:
rushing = pd.read_csv("2021-rushing.csv",header=1)
rushing.head()

,Rk,Player,Tm,Age,Pos,G,GS,Att,Yds,TD,1D,Lng,Y/A,Y/G,Fmb
0,1,Jonathan Taylor*+,IND,22,RB,17,17,332,1811,18,107,83,5.5,106.5,4
1,2,Najee Harris*,PIT,23,RB,17,17,307,1200,7,62,37,3.9,70.6,0
2,3,Joe Mixon*,CIN,25,RB,16,16,292,1205,13,60,32,4.1,75.3,2
3,4,Antonio Gibson,WAS,23,RB,16,14,258,1037,7,65,27,4.0,64.8,6
4,5,Dalvin Cook*,MIN,26,RB,13,13,249,1159,6,57,66,4.7,89.2,3


In [727]:
receiving = pd.read_csv("2021-receiving.csv",header=0)
receiving.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Cooper Kupp*+,LAR,28,WR,17,17,191,145,75.90%,1947,13.4,16,89,59,10.2,8.5,114.5,0
1,2,Davante Adams*+,GNB,29,WR,16,16,169,123,72.80%,1553,12.6,11,84,59,9.2,7.7,97.1,0
2,3,Tyreek Hill*,KAN,27,WR,17,16,159,111,69.80%,1239,11.2,9,75,75,7.8,6.5,72.9,2
3,4,Justin Jefferson*,MIN,22,WR,17,17,167,108,64.70%,1616,15.0,10,75,56,9.7,6.4,95.1,1
4,5,Mark Andrews*+,BAL,26,TE,17,9,153,107,69.90%,1361,12.7,9,75,43,8.9,6.3,80.1,1


# Making a Model on just WRs and TEs to figure out functionality

In [728]:
wrte_rating = madden[(madden["Position"]=="WR") | (madden["Position"]=="TE")]
wrte_rating = wrte_rating.rename(columns={"Full Name":"Player"})

#lets filter this to only players whose rating is above 65
wrte_rating = wrte_rating[wrte_rating["overall_rating"] >= 65]
wrte_rating.head()

,Player,Team,Position,college,awareness_rating,throwPower_rating,kickReturn_rating,leadBlock_rating,strength_rating,bCVision_rating,...,blockShedding_rating,runBlockFinesse_rating,teamId,agility_rating,fullNameForSearch,overall_rating,passBlockFinesse_rating,age,primaryKey,class
1,Davante Adams,Raiders,WR,Fresno State,99,45,45,25,63,94,...,52,12,23,95,Davante Adams,99,17,29,10823,5
4,Cooper Kupp,Rams,WR,Eastern Wash.,99,59,84,34,66,96,...,28,28,24,95,Cooper Kupp,98,34,29,12558,5
6,Travis Kelce,Chiefs,TE,Cincinnati,97,70,6,59,79,92,...,40,66,9,86,Travis Kelce,98,63,32,890,5
9,George Kittle,49ers,TE,Iowa,96,30,31,65,82,90,...,31,71,15,85,George Kittle,97,58,28,12836,5
11,Tyreek Hill,Dolphins,WR,West Alabama,90,41,92,20,64,97,...,21,12,12,98,Tyreek Hill,97,19,28,17838,5


In [729]:
wrte_stats = receiving[(receiving["Pos"]=="WR") | (receiving["Pos"]=="TE")]

#lets get rid of the "+" and "*" from their names so we can join the tables
wrte_stats["Player"] = wrte_stats["Player"].map(lambda x: re.sub(r'[*+]', '', x))
wrte_stats["Ctch%"] = wrte_stats["Ctch%"].map(lambda x: re.sub(r'[%]', '', x))
wrte_stats.head()

/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/3569740911.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrte_stats["Player"] = wrte_stats["Player"].map(lambda x: re.sub(r'[*+]', '', x))
/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/3569740911.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrte_stats["Ctch%"] = wrte_stats["Ctch%"].map(lambda x: re.sub(r'[%]', '', x))


,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Cooper Kupp,LAR,28,WR,17,17,191,145,75.90,1947,13.4,16,89,59,10.2,8.5,114.5,0
1,2,Davante Adams,GNB,29,WR,16,16,169,123,72.80,1553,12.6,11,84,59,9.2,7.7,97.1,0
2,3,Tyreek Hill,KAN,27,WR,17,16,159,111,69.80,1239,11.2,9,75,75,7.8,6.5,72.9,2
3,4,Justin Jefferson,MIN,22,WR,17,17,167,108,64.70,1616,15.0,10,75,56,9.7,6.4,95.1,1
4,5,Mark Andrews,BAL,26,TE,17,9,153,107,69.90,1361,12.7,9,75,43,8.9,6.3,80.1,1


In [730]:
#now lets join the two dataframes on the names

wrte = wrte_rating[["Player", "overall_rating", "class"]].merge(wrte_stats, how="left", on="Player")

In [731]:
wrte = wrte.dropna(subset=['Yds'])
wrte

,Player,overall_rating,class,Rk,Tm,Age,Pos,G,GS,Tgt,...,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,Davante Adams,99,5,2.0,GNB,29.0,WR,16.0,16.0,169.0,...,72.80,1553.0,12.6,11.0,84.0,59.0,9.2,7.7,97.1,0.0
1,Cooper Kupp,98,5,1.0,LAR,28.0,WR,17.0,17.0,191.0,...,75.90,1947.0,13.4,16.0,89.0,59.0,10.2,8.5,114.5,0.0
2,Travis Kelce,98,5,13.0,KAN,32.0,TE,16.0,16.0,134.0,...,68.70,1125.0,12.2,9.0,63.0,69.0,8.4,5.8,70.3,1.0
3,George Kittle,97,5,36.0,SFO,28.0,TE,14.0,14.0,94.0,...,75.50,910.0,12.8,6.0,42.0,48.0,9.7,5.1,65.0,2.0
4,Tyreek Hill,97,5,3.0,KAN,27.0,WR,17.0,16.0,159.0,...,69.80,1239.0,11.2,9.0,75.0,75.0,7.8,6.5,72.9,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330,Eric Saubert,65,1,318.0,DEN,27.0,TE,17.0,4.0,12.0,...,66.70,47.0,5.9,1.0,2.0,11.0,3.9,0.5,2.8,0.0
331,Kylen Granson,65,1,279.0,IND,23.0,TE,17.0,0.0,15.0,...,73.30,106.0,9.6,0.0,4.0,27.0,7.1,0.6,6.2,0.0
332,Noah Gray,65,1,325.0,KAN,22.0,TE,16.0,1.0,10.0,...,70.00,36.0,5.1,1.0,2.0,8.0,3.6,0.4,2.3,0.0
334,Richard Rodgers,65,1,449.0,PHI,29.0,TE,3.0,0.0,2.0,...,100.00,11.0,5.5,0.0,0.0,9.0,5.5,0.7,3.7,0.0


In [732]:
wrte.columns

Index(['Player', 'overall_rating', 'class', 'Rk', 'Tm', 'Age', 'Pos', 'G',
       'GS', 'Tgt', 'Rec', 'Ctch%', 'Yds', 'Y/R', 'TD', '1D', 'Lng', 'Y/Tgt',
       'R/G', 'Y/G', 'Fmb'],
      dtype='object')

# Creating a WRTE model

In [867]:
# first lets set up our X and y dataframes
X = wrte[["Tgt", "Ctch%", "Yds", "Y/R", "TD", "Y/G", "Rec", "Y/Tgt", "R/G"]]
y = wrte[["class"]]

In [868]:
X.head()

,Tgt,Ctch%,Yds,Y/R,TD,Y/G,Rec,Y/Tgt,R/G
0,169.0,72.80,1553.0,12.6,11.0,97.1,123.0,9.2,7.7
1,191.0,75.90,1947.0,13.4,16.0,114.5,145.0,10.2,8.5
2,134.0,68.70,1125.0,12.2,9.0,70.3,92.0,8.4,5.8
3,94.0,75.50,910.0,12.8,6.0,65.0,71.0,9.7,5.1
4,159.0,69.80,1239.0,11.2,9.0,72.9,111.0,7.8,6.5


In [869]:
y = np.ravel(y)
y

array([5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [551]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metri
import lightgbm
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, LeaveOneOut
from sklearn.model_selection import cross_val_score, cross_val_predict


def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["GBM"])
    if classifier_name == "GBM":
        #p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}
        svc_c = trial.suggest_float("learning_rate", .001, .15, log=True)
        svc_c1 = trial.suggest_int("n_estimators", 5, 50, log=True)
        svc_c2= trial.suggest_int("max_depth", 2, 30, log=True)
        #model = GradientBoostingClassifier()
        classifier_obj = GradientBoostingClassifier(random_state=0, learning_rate = svc_c, max_depth=svc_c2, n_estimators=svc_c1)
    cv = LeaveOneOut()
    y_pred = cross_val_predict(classifier_obj, X, y, cv=cv)
    accuracy = np.diag(pd.crosstab(y_pred,y)).sum()/len(y)
    return accuracy


In [871]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-05-06 16:04:12,937] A new study created in memory with name: no-name-15dd947a-1426-4872-a3b4-c7010ce3606e
[I 2023-05-06 16:04:13,372] Trial 0 finished with value: 0.562929292929293 and parameters: {'classifier': 'GBM', 'learning_rate': 0.00470098854949816, 'n_estimators': 12, 'max_depth': 6}. Best is trial 0 with value: 0.562929292929293.
[I 2023-05-06 16:04:13,695] Trial 1 finished with value: 0.562929292929293 and parameters: {'classifier': 'GBM', 'learning_rate': 0.004981537845612498, 'n_estimators': 8, 'max_depth': 12}. Best is trial 0 with value: 0.562929292929293.
[I 2023-05-06 16:04:14,101] Trial 2 finished with value: 0.562929292929293 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0016386978731361406, 'n_estimators': 12, 'max_depth': 6}. Best is trial 0 with value: 0.562929292929293.
[I 2023-05-06 16:04:14,233] Trial 3 finished with value: 0.562929292929293 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0019749536134384045, 'n_estimators': 9, 'max_d

[I 2023-05-06 16:04:21,193] Trial 33 finished with value: 0.6530303030303031 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0909771226053467, 'n_estimators': 8, 'max_depth': 3}. Best is trial 32 with value: 0.6846464646464646.
[I 2023-05-06 16:04:21,302] Trial 34 finished with value: 0.6622222222222222 and parameters: {'classifier': 'GBM', 'learning_rate': 0.10653587284618254, 'n_estimators': 5, 'max_depth': 2}. Best is trial 32 with value: 0.6846464646464646.
[I 2023-05-06 16:04:21,617] Trial 35 finished with value: 0.6303030303030303 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0754701435028722, 'n_estimators': 7, 'max_depth': 10}. Best is trial 32 with value: 0.6846464646464646.
[I 2023-05-06 16:04:21,950] Trial 36 finished with value: 0.6257575757575758 and parameters: {'classifier': 'GBM', 'learning_rate': 0.055559961099565854, 'n_estimators': 9, 'max_depth': 6}. Best is trial 32 with value: 0.6846464646464646.
[I 2023-05-06 16:04:22,188] Trial 37 finished wi

[I 2023-05-06 16:04:29,579] Trial 66 finished with value: 0.6841414141414142 and parameters: {'classifier': 'GBM', 'learning_rate': 0.09643498417409013, 'n_estimators': 14, 'max_depth': 6}. Best is trial 46 with value: 0.6893939393939393.
[I 2023-05-06 16:04:29,943] Trial 67 finished with value: 0.6530303030303031 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0660703935620697, 'n_estimators': 19, 'max_depth': 3}. Best is trial 46 with value: 0.6893939393939393.
[I 2023-05-06 16:04:30,144] Trial 68 finished with value: 0.6668686868686867 and parameters: {'classifier': 'GBM', 'learning_rate': 0.04657662000187692, 'n_estimators': 13, 'max_depth': 2}. Best is trial 46 with value: 0.6893939393939393.
[I 2023-05-06 16:04:30,379] Trial 69 finished with value: 0.671010101010101 and parameters: {'classifier': 'GBM', 'learning_rate': 0.12893471444152008, 'n_estimators': 10, 'max_depth': 4}. Best is trial 46 with value: 0.6893939393939393.
[I 2023-05-06 16:04:30,991] Trial 70 finished 

[I 2023-05-06 16:04:42,222] Trial 99 finished with value: 0.625959595959596 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0940202909199842, 'n_estimators': 17, 'max_depth': 5}. Best is trial 82 with value: 0.6893939393939394.


FrozenTrial(number=82, state=TrialState.COMPLETE, values=[0.6893939393939394], datetime_start=datetime.datetime(2023, 5, 6, 16, 4, 35, 308586), datetime_complete=datetime.datetime(2023, 5, 6, 16, 4, 35, 540344), params={'classifier': 'GBM', 'learning_rate': 0.09559863889869119, 'n_estimators': 16, 'max_depth': 2}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('GBM',)), 'learning_rate': FloatDistribution(high=0.15, log=True, low=0.001, step=None), 'n_estimators': IntDistribution(high=50, log=True, low=5, step=1), 'max_depth': IntDistribution(high=12, log=True, low=2, step=1)}, trial_id=82, value=None)


# All skill positions model

In [736]:
#filter out madden positions
madden = madden[(madden["Position"]=="QB") | (madden["Position"]=="WR") | (madden["Position"]=="RB") | (madden["Position"]=="TE")]


In [737]:
new_columns_p = {column: f"p_{column}" for column in passing.columns}
passing = passing.rename(columns=new_columns_p)

new_columns_rus = {column: f"ru_{column}" for column in rushing.columns}
rushing = rushing.rename(columns=new_columns_rus)

new_columns_rec = {column: f"re_{column}" for column in receiving.columns}
receiving = receiving.rename(columns=new_columns_rec)

passing = passing.rename(columns = {"p_Player" : "Player"})
receiving = receiving.rename(columns = {"re_Player" : "Player"})
rushing = rushing.rename(columns = {"ru_Player" : "Player"})

test = receiving.merge(rushing, on="Player", how="outer")
stats = test.merge(passing, on="Player", how='outer')
stats = stats.fillna(0.0)

stats['ProBowl'] = stats['Player'].str.contains('\*')
stats['AllPro'] = stats['Player'].str.contains('\+')

stats['Player'] = stats['Player'].str.replace('*', '')
stats['Player'] = stats['Player'].str.replace('+', '')



/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/266377572.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stats['Player'] = stats['Player'].str.replace('*', '')
/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/266377572.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stats['Player'] = stats['Player'].str.replace('+', '')


In [738]:
stats["Player"] = stats["Player"].map(lambda x: re.sub(r' Jr.', '', x))
madden["Player"] = madden["Player"].map(lambda x: re.sub(r' Jr', '', x))
stats2 = stats[(stats["p_Pos"]=="QB") | (stats["re_Pos"]=="WR") | (stats["ru_Pos"]=="RB") | (stats["re_Pos"]=="TE")]

In [739]:
combined = stats2.merge(madden[["Player", "class", "overall_rating"]], how="left", on="Player")
combined2 = combined.dropna(subset=["overall_rating"])
combined2["re_Ctch%"] = combined2["re_Ctch%"].map(lambda x: re.sub(r'[%]', '', x) if isinstance(x,str) else x)
combined2 = combined2[(combined2["overall_rating"]>=65)]

/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/1420688824.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined2["re_Ctch%"] = combined2["re_Ctch%"].map(lambda x: re.sub(r'[%]', '', x) if isinstance(x,str) else x)


In [740]:
combined2.shape

(278, 67)

In [785]:
combined2["GS"] = combined2[["re_GS","ru_GS", "p_GS"]].max(axis=1)
combined2["G"] = combined2[["re_G","ru_G", "p_G"]].max(axis=1)

In [741]:
combined2.columns

Index(['re_Rk', 'Player', 're_Tm', 're_Age', 're_Pos', 're_G', 're_GS',
       're_Tgt', 're_Rec', 're_Ctch%', 're_Yds', 're_Y/R', 're_TD', 're_1D',
       're_Lng', 're_Y/Tgt', 're_R/G', 're_Y/G', 're_Fmb', 'ru_Rk', 'ru_Tm',
       'ru_Age', 'ru_Pos', 'ru_G', 'ru_GS', 'ru_Att', 'ru_Yds', 'ru_TD',
       'ru_1D', 'ru_Lng', 'ru_Y/A', 'ru_Y/G', 'ru_Fmb', 'p_Rk', 'p_Tm',
       'p_Age', 'p_Pos', 'p_G', 'p_GS', 'p_QBrec', 'p_Cmp', 'p_Att', 'p_Cmp%',
       'p_Yds', 'p_TD', 'p_TD%', 'p_Int', 'p_Int%', 'p_1D', 'p_Lng', 'p_Y/A',
       'p_AY/A', 'p_Y/C', 'p_Y/G', 'p_Rate', 'p_QBR', 'p_Sk', 'p_Yds.1',
       'p_Sk%', 'p_NY/A', 'p_ANY/A', 'p_4QC', 'p_GWD', 'ProBowl', 'AllPro',
       'class', 'overall_rating'],
      dtype='object')

In [909]:
#create the X and y values
X = combined2[["GS",'re_Tgt', 're_Rec', 're_Ctch%', 're_Yds', 're_Y/R', 're_TD', 're_1D',
       're_Lng', 're_Y/Tgt', 're_R/G', 're_Y/G', 're_Fmb', 'ru_Att', 'ru_Yds', 'ru_TD',
       'ru_1D', 'ru_Lng', 'ru_Y/A', 'ru_Y/G', 'ru_Fmb', 'p_Att', 'p_Cmp%',
       'p_Yds', 'p_TD', 'p_TD%', 'p_Int', 'p_Int%', 'p_1D', 'p_Lng', 'p_Y/A',
        'p_Y/C', 'p_Y/G', 'p_Rate', 'p_QBR', 'p_NY/A', 'ProBowl', 'AllPro']]
    
#X = combined2[["re_GS",'re_TD','re_R/G', 're_Y/G',"re_Tgt",
 #              'ru_Yds', 'ru_TD','ru_Y/A', 'ru_Y/G',
  #             'p_Cmp%', 'p_Yds', 'p_TD','p_Int', 'p_Y/G','p_QBR',
   #             ]]

#X = combined2[["GS",'ru_Att', 'ru_Yds', 'ru_TD',
 #      'ru_1D', 'ru_Lng', 'ru_Y/A', 'ru_Y/G', 'ru_Fmb', 'p_Att', 'p_Cmp%',
  #     'p_Yds', 'p_TD', 'p_TD%', 'p_Int', 'p_Int%', 'p_1D', 'p_Lng', 'p_Y/A',
   #     'p_Y/C', 'p_Y/G', 'p_Rate', 'p_QBR', 'p_NY/A', 'ProBowl', 'AllPro']]
    

In [910]:
X.head()

,GS,re_Tgt,re_Rec,re_Ctch%,re_Yds,re_Y/R,re_TD,re_1D,re_Lng,re_Y/Tgt,...,p_1D,p_Lng,p_Y/A,p_Y/C,p_Y/G,p_Rate,p_QBR,p_NY/A,ProBowl,AllPro
0,17.0,191.0,145.0,75.90,1947.0,13.4,16.0,89.0,59.0,10.2,...,0.0,0.0,0.0,0.0,0.0,39.6,10.4,0.00,True,True
1,16.0,169.0,123.0,72.80,1553.0,12.6,11.0,84.0,59.0,9.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,True,True
2,16.0,159.0,111.0,69.80,1239.0,11.2,9.0,75.0,75.0,7.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,True,False
3,17.0,167.0,108.0,64.70,1616.0,15.0,10.0,75.0,56.0,9.7,...,2.0,24.0,8.8,17.5,2.1,80.2,42.7,8.75,True,False
4,9.0,153.0,107.0,69.90,1361.0,12.7,9.0,75.0,43.0,8.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,True,True


In [911]:
y = combined2[["class"]]
y = np.ravel(y)
y

array([5., 5., 5., 4., 4., 3., 4., 3., 5., 3., 4., 3., 5., 3., 3., 2., 3.,
       2., 3., 2., 3., 2., 2., 4., 4., 3., 4., 3., 4., 3., 3., 2., 4., 5.,
       4., 2., 3., 3., 3., 2., 3., 3., 2., 2., 3., 4., 2., 1., 2., 2., 3.,
       2., 3., 2., 4., 2., 2., 2., 3., 2., 3., 2., 1., 3., 2., 2., 1., 1.,
       2., 1., 1., 2., 1., 3., 2., 2., 5., 1., 2., 2., 2., 2., 2., 2., 1.,
       2., 2., 2., 1., 2., 2., 1., 2., 3., 2., 2., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 3., 1., 1., 1., 1., 1., 1., 2., 2., 1., 1., 2., 1., 2.,
       2., 2., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 2., 2., 1., 1.,
       1., 1., 1., 2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [870]:
from sklearn.model_selection import cross_val_score, KFold

def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", ["GBM"])
    if classifier_name == "GBM":
        #p_test3 = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 'n_estimators':[100,250,500,750,1000,1250,1500,1750]}
        svc_c = trial.suggest_float("learning_rate", .001, .15, log=True)
        svc_c1 = trial.suggest_int("n_estimators", 5, 50, log=True)
        svc_c2= trial.suggest_int("max_depth", 2, 12, log=True)
        #model = GradientBoostingClassifier()
        classifier_obj = GradientBoostingClassifier(random_state=0, learning_rate = svc_c, max_depth=svc_c2, n_estimators=svc_c1)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(classifier_obj, X, y, cv=kf, scoring='accuracy')
    # Calculate mean accuracy across the folds
    accuracy = np.mean(scores)
    return accuracy


In [763]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-05-06 15:36:48,216] A new study created in memory with name: no-name-073d1200-c01e-4061-b8a9-8e1dcbe5ccd3
[I 2023-05-06 15:36:49,310] Trial 0 finished with value: 0.6688961038961039 and parameters: {'classifier': 'GBM', 'learning_rate': 0.05376583816691311, 'n_estimators': 43, 'max_depth': 3}. Best is trial 0 with value: 0.6688961038961039.
[I 2023-05-06 15:36:49,422] Trial 1 finished with value: 0.6833116883116883 and parameters: {'classifier': 'GBM', 'learning_rate': 0.14423963569006115, 'n_estimators': 5, 'max_depth': 2}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:36:50,972] Trial 2 finished with value: 0.6005844155844156 and parameters: {'classifier': 'GBM', 'learning_rate': 0.020734229153939455, 'n_estimators': 21, 'max_depth': 10}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:36:52,084] Trial 3 finished with value: 0.5647402597402598 and parameters: {'classifier': 'GBM', 'learning_rate': 0.002673265769821407, 'n_estimators': 35, '

[I 2023-05-06 15:37:00,626] Trial 33 finished with value: 0.6725974025974025 and parameters: {'classifier': 'GBM', 'learning_rate': 0.14650326014317788, 'n_estimators': 6, 'max_depth': 4}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:37:00,753] Trial 34 finished with value: 0.632922077922078 and parameters: {'classifier': 'GBM', 'learning_rate': 0.09781848382816616, 'n_estimators': 5, 'max_depth': 2}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:37:00,952] Trial 35 finished with value: 0.6185714285714285 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0605811457389501, 'n_estimators': 7, 'max_depth': 3}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:37:01,810] Trial 36 finished with value: 0.6655194805194806 and parameters: {'classifier': 'GBM', 'learning_rate': 0.11648241185153237, 'n_estimators': 45, 'max_depth': 2}. Best is trial 1 with value: 0.6833116883116883.
[I 2023-05-06 15:37:02,935] Trial 37 finished with va

[I 2023-05-06 15:37:13,811] Trial 66 finished with value: 0.6437662337662338 and parameters: {'classifier': 'GBM', 'learning_rate': 0.10713590833529829, 'n_estimators': 5, 'max_depth': 2}. Best is trial 49 with value: 0.6833766233766234.
[I 2023-05-06 15:37:14,470] Trial 67 finished with value: 0.607987012987013 and parameters: {'classifier': 'GBM', 'learning_rate': 0.12288205608630777, 'n_estimators': 7, 'max_depth': 8}. Best is trial 49 with value: 0.6833766233766234.
[I 2023-05-06 15:37:14,877] Trial 68 finished with value: 0.6653896103896104 and parameters: {'classifier': 'GBM', 'learning_rate': 0.06453077796260516, 'n_estimators': 12, 'max_depth': 4}. Best is trial 49 with value: 0.6833766233766234.
[I 2023-05-06 15:37:15,499] Trial 69 finished with value: 0.676038961038961 and parameters: {'classifier': 'GBM', 'learning_rate': 0.07454154048862749, 'n_estimators': 24, 'max_depth': 3}. Best is trial 49 with value: 0.6833766233766234.
[I 2023-05-06 15:37:16,336] Trial 70 finished wi

[I 2023-05-06 15:37:27,216] Trial 99 finished with value: 0.6546753246753247 and parameters: {'classifier': 'GBM', 'learning_rate': 0.11701925507819125, 'n_estimators': 16, 'max_depth': 5}. Best is trial 85 with value: 0.6905844155844156.


FrozenTrial(number=85, state=TrialState.COMPLETE, values=[0.6905844155844156], datetime_start=datetime.datetime(2023, 5, 6, 15, 37, 21, 174053), datetime_complete=datetime.datetime(2023, 5, 6, 15, 37, 21, 449885), params={'classifier': 'GBM', 'learning_rate': 0.13461152192596762, 'n_estimators': 13, 'max_depth': 2}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('GBM',)), 'learning_rate': FloatDistribution(high=0.15, log=True, low=0.001, step=None), 'n_estimators': IntDistribution(high=50, log=True, low=5, step=1), 'max_depth': IntDistribution(high=12, log=True, low=2, step=1)}, trial_id=85, value=None)


In [912]:
#best parameters
lr = 0.13461152192596762
ne = 13
md = 2

In [913]:
classifier_obj = GradientBoostingClassifier(random_state=0, learning_rate = lr, max_depth=md, n_estimators=ne)

cv = KFold(n_splits=5, shuffle=True, random_state=42)
y_pred = cross_val_predict(classifier_obj, X, y, cv=cv)
classifier_obj.fit(X,y)



GradientBoostingClassifier(learning_rate=0.13461152192596762, max_depth=2,
                           n_estimators=13, random_state=0)

In [914]:
pd.crosstab(y,y_pred)

col_0,1.0,2.0,3.0,4.0,5.0
row_0,,,,,
1.0,145,11,1,0,0
2.0,28,26,7,1,0
3.0,3,14,11,4,2
4.0,0,2,9,3,1
5.0,0,1,4,1,4


In [915]:
accuracy = np.diag(pd.crosstab(y_pred,y)).sum()/len(y)
print(accuracy)

0.6798561151079137


In [922]:
df_empty = pd.DataFrame(columns=X.columns)

In [923]:
df_empty.columns

Index(['GS', 're_Tgt', 're_Rec', 're_Ctch%', 're_Yds', 're_Y/R', 're_TD',
       're_1D', 're_Lng', 're_Y/Tgt', 're_R/G', 're_Y/G', 're_Fmb', 'ru_Att',
       'ru_Yds', 'ru_TD', 'ru_1D', 'ru_Lng', 'ru_Y/A', 'ru_Y/G', 'ru_Fmb',
       'p_Att', 'p_Cmp%', 'p_Yds', 'p_TD', 'p_TD%', 'p_Int', 'p_Int%', 'p_1D',
       'p_Lng', 'p_Y/A', 'p_Y/C', 'p_Y/G', 'p_Rate', 'p_QBR', 'p_NY/A',
       'ProBowl', 'AllPro'],
      dtype='object')

In [924]:
lamar = {'GS':15, 're_Tgt':0, 're_Rec':0, 're_Ctch%':0, 're_Yds':0, 're_Y/R':0, 're_TD':0,
       're_1D':0, 're_Lng':0, 're_Y/Tgt':0, 're_R/G':0, 're_Y/G':0, 're_Fmb':0, 'ru_Att':176,
       'ru_Yds':1206, 'ru_TD':7, 'ru_1D':71, 'ru_Lng':47, 'ru_Y/A':6.9, 'ru_Y/G':80.4, 'ru_Fmb':6,
         'p_Att':401, 'p_Cmp%':66.1, 'p_Yds':3127, 'p_TD':36, 'p_TD%':9.0, 'p_Int':6, 'p_Int%':1.5,
       'p_1D':161, 'p_Lng':83, 'p_Y/A':7.8, 'p_Y/C':11.8, 'p_Y/G':208.5, 'p_Rate':113.3, 'p_QBR':83.0, 'p_NY/A':7.13,
       'ProBowl':True, 'AllPro':True}

# append the new row to the dataframe
df_empty = df_empty.append(lamar, ignore_index=True)

/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/2792457214.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_empty = df_empty.append(lamar, ignore_index=True)


In [925]:
df_empty

,GS,re_Tgt,re_Rec,re_Ctch%,re_Yds,re_Y/R,re_TD,re_1D,re_Lng,re_Y/Tgt,...,p_1D,p_Lng,p_Y/A,p_Y/C,p_Y/G,p_Rate,p_QBR,p_NY/A,ProBowl,AllPro
0,15,0,0,0,0,0,0,0,0,0,...,161,83,7.8,11.8,208.5,113.3,83.0,7.13,True,True


In [926]:
classifier_obj.predict(df_empty)

array([1.])

# WRTE model

In [872]:
rec = combined2[(combined2["re_Pos"]=="WR") | (combined2["re_Pos"]=="TE")]


In [873]:
rec

,re_Rk,Player,re_Tm,re_Age,re_Pos,re_G,re_GS,re_Tgt,re_Rec,re_Ctch%,...,p_NY/A,p_ANY/A,p_4QC,p_GWD,ProBowl,AllPro,class,overall_rating,GS,G
0,1.0,Cooper Kupp,LAR,28.0,WR,17.0,17.0,191.0,145.0,75.90,...,0.00,0.00,0.0,0.0,True,True,5.0,98.0,17.0,17.0
1,2.0,Davante Adams,GNB,29.0,WR,16.0,16.0,169.0,123.0,72.80,...,0.00,0.00,0.0,0.0,True,True,5.0,99.0,16.0,16.0
2,3.0,Tyreek Hill,KAN,27.0,WR,17.0,16.0,159.0,111.0,69.80,...,0.00,0.00,0.0,0.0,True,False,5.0,97.0,16.0,17.0
3,4.0,Justin Jefferson,MIN,22.0,WR,17.0,17.0,167.0,108.0,64.70,...,8.75,8.75,0.0,0.0,True,False,4.0,93.0,17.0,17.0
4,5.0,Mark Andrews,BAL,26.0,TE,17.0,9.0,153.0,107.0,69.90,...,0.00,0.00,0.0,0.0,True,True,4.0,93.0,9.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,476.0,Davion Davis,HOU,25.0,WR,2.0,1.0,3.0,1.0,33.30,...,0.00,0.00,0.0,0.0,False,False,1.0,65.0,1.0,2.0
456,478.0,Marvin Hall,ATL,28.0,WR,2.0,0.0,1.0,1.0,100.00,...,0.00,0.00,0.0,0.0,False,False,1.0,68.0,0.0,2.0
461,484.0,Andy Isabella,ARI,25.0,WR,8.0,0.0,1.0,1.0,100.00,...,0.00,0.00,0.0,0.0,False,False,1.0,70.0,0.0,8.0
465,489.0,Hunter Long,MIA,23.0,TE,7.0,2.0,3.0,1.0,33.30,...,0.00,0.00,0.0,0.0,False,False,1.0,66.0,2.0,7.0


In [904]:
# set up X and y
X = rec[["GS",'re_Tgt', 're_Rec', 're_Ctch%', 're_Yds', 're_Y/R', 're_TD', 're_1D',
       're_Lng', 're_Y/Tgt', 're_R/G', 're_Y/G']]
X.head()

,GS,re_Tgt,re_Rec,re_Ctch%,re_Yds,re_Y/R,re_TD,re_1D,re_Lng,re_Y/Tgt,re_R/G,re_Y/G
0,17.0,191.0,145.0,75.90,1947.0,13.4,16.0,89.0,59.0,10.2,8.5,114.5
1,16.0,169.0,123.0,72.80,1553.0,12.6,11.0,84.0,59.0,9.2,7.7,97.1
2,16.0,159.0,111.0,69.80,1239.0,11.2,9.0,75.0,75.0,7.8,6.5,72.9
3,17.0,167.0,108.0,64.70,1616.0,15.0,10.0,75.0,56.0,9.7,6.4,95.1
4,9.0,153.0,107.0,69.90,1361.0,12.7,9.0,75.0,43.0,8.9,6.3,80.1


In [905]:
y = rec[["class"]]
y = np.ravel(y)
y[:5]

array([5., 5., 5., 4., 4.])

In [877]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-05-06 16:09:08,484] A new study created in memory with name: no-name-f9b59e39-a452-4ae1-9ad9-e81ccb7992f8
[I 2023-05-06 16:09:09,260] Trial 0 finished with value: 0.5666049953746531 and parameters: {'classifier': 'GBM', 'learning_rate': 0.001602879007243192, 'n_estimators': 25, 'max_depth': 5}. Best is trial 0 with value: 0.5666049953746531.
[I 2023-05-06 16:09:09,530] Trial 1 finished with value: 0.5666049953746531 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0032979888326516785, 'n_estimators': 6, 'max_depth': 8}. Best is trial 0 with value: 0.5666049953746531.
[I 2023-05-06 16:09:11,242] Trial 2 finished with value: 0.6525439407955597 and parameters: {'classifier': 'GBM', 'learning_rate': 0.07417143951985888, 'n_estimators': 29, 'max_depth': 9}. Best is trial 2 with value: 0.6525439407955597.
[I 2023-05-06 16:09:12,232] Trial 3 finished with value: 0.5666049953746531 and parameters: {'classifier': 'GBM', 'learning_rate': 0.0020413681242347223, 'n_estimators': 41,

[I 2023-05-06 16:09:28,002] Trial 33 finished with value: 0.6652173913043479 and parameters: {'classifier': 'GBM', 'learning_rate': 0.023498657371227593, 'n_estimators': 23, 'max_depth': 4}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:28,491] Trial 34 finished with value: 0.6483811285846439 and parameters: {'classifier': 'GBM', 'learning_rate': 0.01572519097374778, 'n_estimators': 29, 'max_depth': 2}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:30,471] Trial 35 finished with value: 0.6613320999074931 and parameters: {'classifier': 'GBM', 'learning_rate': 0.06588339408996126, 'n_estimators': 37, 'max_depth': 8}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:31,035] Trial 36 finished with value: 0.7080481036077705 and parameters: {'classifier': 'GBM', 'learning_rate': 0.03149892600214174, 'n_estimators': 29, 'max_depth': 3}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:31,927] Trial 37 finish

[I 2023-05-06 16:09:50,700] Trial 66 finished with value: 0.6567067530064755 and parameters: {'classifier': 'GBM', 'learning_rate': 0.027624008240845984, 'n_estimators': 22, 'max_depth': 5}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:51,158] Trial 67 finished with value: 0.6996299722479185 and parameters: {'classifier': 'GBM', 'learning_rate': 0.04464964756913781, 'n_estimators': 29, 'max_depth': 2}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:51,673] Trial 68 finished with value: 0.6908418131359852 and parameters: {'classifier': 'GBM', 'learning_rate': 0.01992739742963853, 'n_estimators': 25, 'max_depth': 3}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:52,183] Trial 69 finished with value: 0.6954671600370028 and parameters: {'classifier': 'GBM', 'learning_rate': 0.026162092258183584, 'n_estimators': 33, 'max_depth': 2}. Best is trial 23 with value: 0.7125809435707678.
[I 2023-05-06 16:09:53,759] Trial 70 finis

[I 2023-05-06 16:10:14,743] Trial 99 finished with value: 0.7083256244218316 and parameters: {'classifier': 'GBM', 'learning_rate': 0.03256025740723384, 'n_estimators': 30, 'max_depth': 2}. Best is trial 23 with value: 0.7125809435707678.


FrozenTrial(number=23, state=TrialState.COMPLETE, values=[0.7125809435707678], datetime_start=datetime.datetime(2023, 5, 6, 16, 9, 22, 442013), datetime_complete=datetime.datetime(2023, 5, 6, 16, 9, 22, 772421), params={'classifier': 'GBM', 'learning_rate': 0.046651492919920205, 'n_estimators': 20, 'max_depth': 2}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'classifier': CategoricalDistribution(choices=('GBM',)), 'learning_rate': FloatDistribution(high=0.15, log=True, low=0.001, step=None), 'n_estimators': IntDistribution(high=50, log=True, low=5, step=1), 'max_depth': IntDistribution(high=12, log=True, low=2, step=1)}, trial_id=23, value=None)


In [900]:
#best parameters
lr = 0.046651492919920205
ne = 20
md = 2

In [906]:
classifier_obj = GradientBoostingClassifier(random_state=0, learning_rate = lr, max_depth=md, n_estimators=ne)

cv = KFold(n_splits=5, shuffle=True, random_state=42)
y_pred = cross_val_predict(classifier_obj, X, y, cv=cv)
classifier_obj.fit(X,y)



GradientBoostingClassifier(learning_rate=0.046651492919920205, max_depth=2,
                           n_estimators=20, random_state=0)

In [907]:
pd.crosstab(y,y_pred)

col_0,1.0,2.0,3.0,4.0,5.0
row_0,,,,,
1.0,121,11,0,0,0
2.0,22,28,5,0,0
3.0,1,12,11,3,0
4.0,0,2,6,3,1
5.0,0,1,3,0,3


In [908]:
accuracy = np.diag(pd.crosstab(y_pred,y)).sum()/len(y)
print(accuracy)

0.7124463519313304


In [882]:
df_empty = pd.DataFrame(columns=X.columns)

In [883]:
df_empty.columns

Index(['GS', 're_Tgt', 're_Rec', 're_Ctch%', 're_Yds', 're_Y/R', 're_TD',
       're_1D', 're_Lng', 're_Y/Tgt', 're_R/G', 're_Y/G', 're_Fmb', 'ProBowl',
       'AllPro'],
      dtype='object')

In [884]:
mt = {'GS':15, 're_Tgt':185, 're_Rec':149, 're_Ctch%':80.5, 're_Yds':1725, 're_Y/R':11.6, 're_TD':9,
       're_1D':91, 're_Lng':49, 're_Y/Tgt':9.3, 're_R/G':9.3, 're_Y/G':107.8, 're_Fmb':1, 'ProBowl':True,
       'AllPro':True}

# append the new row to the dataframe
df_empty = df_empty.append(mt, ignore_index=True)


/var/folders/pr/kb1ft_x55nzff0pnjrpnykqw0000gn/T/ipykernel_11438/3306045445.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_empty = df_empty.append(mt, ignore_index=True)


In [885]:
classifier_obj.predict(df_empty)

array([4.])